<a href="https://colab.research.google.com/github/talalba/ML_exercises-/blob/master/prob_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import the necessary packages
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.io import imread
from sklearn.model_selection import KFold
from pandas import read_excel
import cv2
import pathlib
import multiprocessing
import importlib
import locale
from random import randrange
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive
# drive.mount('/content/drive')
import gzip
from numpy.linalg import norm
from numpy import dot
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [9]:
data_path = '/content/drive/My Drive/exercise/'

# load the data
data = pd.read_csv(data_path+'bank_data.csv',sep=',')
display(data)
print(data.dtypes)

,id,age,income,children,gender,region,married,car,savings_acct,current_acct,mortgage,pep
0,ID12101,48,17546.00,1,FEMALE,INNER_CITY,NO,NO,NO,NO,NO,YES
1,ID12102,40,30085.10,3,MALE,TOWN,YES,YES,NO,YES,YES,NO
2,ID12103,51,16575.40,0,FEMALE,INNER_CITY,YES,YES,YES,YES,NO,NO
3,ID12104,23,20375.40,3,FEMALE,TOWN,YES,NO,NO,YES,NO,NO
4,ID12105,57,50576.30,0,FEMALE,RURAL,YES,NO,YES,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...
595,ID12696,61,47025.00,2,FEMALE,INNER_CITY,NO,YES,YES,YES,YES,NO
596,ID12697,30,9672.25,0,FEMALE,INNER_CITY,YES,YES,YES,YES,NO,NO
597,ID12698,31,15976.30,0,FEMALE,TOWN,YES,YES,YES,NO,NO,YES
598,ID12699,29,14711.80,0,MALE,INNER_CITY,YES,NO,YES,NO,YES,NO


id               object
age               int64
income          float64
children          int64
gender           object
region           object
married          object
car              object
savings_acct     object
current_acct     object
mortgage         object
pep              object
dtype: object


In [36]:
data2 = data.drop('id',axis=1)
# create dummy variables
le = LabelEncoder()


# Categorical boolean mask
categorical_feature_mask = data2.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = data2.columns[categorical_feature_mask].tolist()

# apply le on categorical feature columns
data2[categorical_cols] = data2[categorical_cols].apply(lambda col: le.fit_transform(col))

display(data2)



,age,income,children,gender,region,married,car,savings_acct,current_acct,mortgage,pep
0,48,17546.00,1,0,0,0,0,0,0,0,1
1,40,30085.10,3,1,3,1,1,0,1,1,0
2,51,16575.40,0,0,0,1,1,1,1,0,0
3,23,20375.40,3,0,3,1,0,0,1,0,0
4,57,50576.30,0,0,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
595,61,47025.00,2,0,0,0,1,1,1,1,0
596,30,9672.25,0,0,0,1,1,1,1,0,0
597,31,15976.30,0,0,3,1,1,1,0,0,1
598,29,14711.80,0,1,0,1,0,1,0,1,0


(600, 10)

In [95]:
data3 = data2.drop('pep',axis=1)
labels = np.array(data2.pep)
# split data into train and test set

# normalize data

cs = MinMaxScaler()
data3 = cs.fit_transform(data3)

data3.shape
no_runs = 50

actual_labels = []
predicted_labels = []

predicted_labels_dt = []
predicted_labels_gnb = []

for ii in range(0,no_runs):

  split = train_test_split(data2,labels, test_size=0.2)
  (X_train, X_test, Y_train, Y_test) = split

  # neigh = KNeighborsClassifier(n_neighbors=12, weights='distance')
  neigh = KNeighborsClassifier(n_neighbors=10)
  neigh.fit(X_train, Y_train)

  # decison tree
  dt = DecisionTreeClassifier(criterion='entropy')
  dt.fit(X_train, Y_train)
  predicted_labels_dt = np.append(predicted_labels_dt,dt.predict(X_test))

  # naive bayes
  gnb = GaussianNB()
  gnb.fit(X_train, Y_train)
  predicted_labels_gnb = np.append(predicted_labels_gnb,gnb.predict(X_test))

  predicted_labels = np.append(predicted_labels,neigh.predict(X_test))
  actual_labels = np.append(actual_labels,Y_test)

print("KNN results")

# print classification performance 
cr = classification_report(actual_labels,predicted_labels)
print(cr)

# print confusion matrix
cm = confusion_matrix(actual_labels, predicted_labels)
print(cm)

# without weights workd better and best value of K is 12

print("GNB results")
# print classification performance 
cr = classification_report(actual_labels,predicted_labels_gnb)
print(cr)

# print confusion matrix
cm = confusion_matrix(actual_labels, predicted_labels_gnb)
print(cm)


print("DT results")
# print classification performance 
cr = classification_report(actual_labels,predicted_labels_dt)
print(cr)

# print confusion matrix
cm = confusion_matrix(actual_labels, predicted_labels_dt)
print(cm)





KNN results
              precision    recall  f1-score   support

         0.0       0.59      0.73      0.65      3280
         1.0       0.54      0.38      0.44      2720

    accuracy                           0.57      6000
   macro avg       0.56      0.55      0.55      6000
weighted avg       0.56      0.57      0.56      6000

[[2393  887]
 [1690 1030]]
GNB results
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3280
         1.0       1.00      1.00      1.00      2720

    accuracy                           1.00      6000
   macro avg       1.00      1.00      1.00      6000
weighted avg       1.00      1.00      1.00      6000

[[3276    4]
 [   0 2720]]
DT results
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      3280
         1.0       1.00      1.00      1.00      2720

    accuracy                           1.00      6000
   macro avg       1.00      1.00      

In [80]:
# sklearn cross valiadtion module is deprecated so using traintest split
# naive bayes
gnb = GaussianNB()
gnb.fit(X_train, Y_train)
predicted_labels_gnb = gnb.predict(X_test)

# decison tree
dt = DecisionTreeClassifier(criterion='entropy')
dt.fit(X_train, Y_train)
predicted_labels_dt = dt.predict(X_test)


# print classification performance 
cr = classification_report(Y_test,predicted_labels_dt)
print(cr)

# print confusion matrix
cm = confusion_matrix(Y_test, predicted_labels_dt)
print(cm)


# print classification performance 
cr = classification_report(Y_test,predicted_labels_gnb)
print(cr)

# print confusion matrix
cm = confusion_matrix(Y_test, predicted_labels_gnb)
print(cm)




              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69
           1       1.00      1.00      1.00        51

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120

[[69  0]
 [ 0 51]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        69
           1       1.00      1.00      1.00        51

    accuracy                           1.00       120
   macro avg       1.00      1.00      1.00       120
weighted avg       1.00      1.00      1.00       120

[[69  0]
 [ 0 51]]


KNN has high variance, whereas deceion tree and naive bayes gaussian classifier have higher bias 